## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


## загрузка данных

In [1]:
# !pip install pyconll

In [11]:
import pyconll

In [12]:
!mkdir datasets

mkdir: cannot create directory ‘datasets’: File exists


In [4]:
# !wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
# !wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-09-19 19:51:42--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81039282 (77M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train.conllu’

./datasets/ru_synta 100%[===================>]  77.28M  6.99MB/s    in 11s     

2021-09-19 19:51:57 (6.74 MB/s) - ‘./datasets/ru_syntagrus-ud-train.conllu’ saved [81039282/81039282]

--2021-09-19 19:51:57--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercon

In [13]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [14]:
def get_data(dataset):
    result = []
    for sent in dataset:
        for token in sent:
            result.append((token.form.lower(), token.upos))
    return result

In [15]:
train_data = get_data(full_train)
train_data[:5]

[('анкета', 'NOUN'),
 ('.', 'PUNCT'),
 ('начальник', 'NOUN'),
 ('областного', 'ADJ'),
 ('управления', 'NOUN')]

In [16]:
test_data = get_data(full_test) 
test_data[:5]

[('алгоритм', 'NOUN'),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN')]

In [17]:
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

In [39]:
tts = 0.25 # train_test_split
train_data_tts = [train_data[:int(len(train_data) * tts)]]

In [26]:
unigram_tagger = UnigramTagger(train_data_tts)

In [27]:
unigram_tagger.evaluate([test_data])

0.8077208236443906

In [28]:
bigram_tagger = BigramTagger(train_data_tts)

In [29]:
bigram_tagger.evaluate([test_data])

0.5753715498938429

In [32]:
trigram_tagger = TrigramTagger(train_data_tts, backoff=bigram_tagger)

In [33]:
trigram_tagger.evaluate([test_data])

0.5747396623192802

#### Комбинация тэггеров

Примущество Backoff Tagging в том, что если текущий тэггер не знает, как тэггировать слово, он передает это следующему и так далее, пока не пройдет перебор по всем тэггерам. В данному случае тэггирование производит последовательность UnigramTagger, BigramTagger, TrigramTagger. Комбинация тэггеров дала немного лучший результат, чем UnigramTagger, BigramTagger по отдельности.

In [19]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

In [38]:
backoff = DefaultTagger('NN') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
tag.evaluate(test_data)

ValueError: too many values to unpack (expected 2)

#### 2. Написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов

In [38]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [39]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
all_test_texts = [' '.join(token.form for token in sent) for sent in full_test]

all_train_labels = [' '.join(token.form for token in sent) for sent in full_train]
all_test_labels = [' '.join(token.form for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [80]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [81]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [82]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [83]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [84]:
test_enc_labels = le.transform(test_label)

In [85]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [86]:
hvectorizer = HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=100)

In [87]:
X_train = hvectorizer.fit_transform(train_tok)

In [88]:
X_test = hvectorizer.transform(test_tok)

In [50]:
X_train.shape

(871526, 100)

In [89]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train, train_enc_labels)

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [90]:
pred = lr.predict(X_test)

In [91]:
accuracy_score(test_enc_labels, pred)

0.686221480807468

In [92]:
classifier = LogisticRegression()

In [93]:
def get_model_accuracy(model):
    X_train = model.fit_transform(train_tok)
    X_test = model.transform(test_tok)
    
    classifier.fit(X_train, train_enc_labels)
    pred = classifier.predict(X_test)
    
    return accuracy_score(test_enc_labels, pred)    

In [94]:
count_vectorizer_char = CountVectorizer(ngram_range=(1,5), analyzer='char', max_features=100)
accuracy_score_ = get_model_accuracy(count_vectorizer_char)
print(f'count_vectorizer_char {accuracy_score_}')

count_vectorizer_char 0.7770195126883025


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [95]:
model = CountVectorizer(ngram_range=(1,5), analyzer='word', max_features=100)
accuracy_score_ = get_model_accuracy(model)
print(f'count_vectorizer_word {accuracy_score_}')

count_vectorizer_word 0.3820982037542547


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [96]:
model = TfidfVectorizer(ngram_range=(1,5),analyzer='char', max_features=100)
accuracy_score_ = get_model_accuracy(model)
print(f'TfidfVectorizer_char {accuracy_score_}')

TfidfVectorizer_char 0.7767077814848515


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [97]:
model = TfidfVectorizer(ngram_range=(1,5),analyzer='word', max_features=100)
accuracy_score_ = get_model_accuracy(model)
print(f'TfidfVectorizer_word {accuracy_score_}')

TfidfVectorizer_word 0.3820897785865939


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Выводы: Accuracy выше всего у UnigramTagger - 0.8077, bigram_tagger и trigram_tagger рядом ~ 0.575, с использованием векторайзеров лучшим оказался count_vectorizer_char - 0.777, на 1 тысячную хуже оказался TfidfVectorizer_char - 0.776, по значениям слов лучший рез-т показал count_vectorizer_word 0.38209 рядом TfidfVectorizer_word 0.38208

# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
* передаём в сетку токен и его соседей
* передаём в сетку только токен

4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [ ]:
# !pip install corus

     |████████████████████████████████| 92kB 7.0MB/s 


In [2]:
import corus

In [3]:
# !wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2021-09-20 20:11:40--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 80.240.100.4, 80.240.100.4
Connecting to www.labinform.ru (www.labinform.ru)|80.240.100.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   828KB/s    in 2.2s    

2021-09-20 20:11:43 (828 KB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [4]:
# !unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [5]:
!rm collection5.zip

In [ ]:
!ls

In [21]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)

Ne5Markup(
    id='467',
    text='Киршнер вступила в должность президента Аргентины на второй срок\r\n\r\nЦеремония принятия присяги избранной президентом Аргентины на второй четырехлетний срок подряд Кристиной Фернандес де Киршнер (Cristina Fernandez de Kirchner) состоялась в субботу в столице страны Буэнос-Айресе, передает аргентинское издание Clarin.\r\n\r\nГлава государства появилась на церемонии в траурной одежде в память о скоропостижно скончавшемся в октябре прошлого года супруге Несторе Киршнере, бывшем президенте страны в 2003-2007 годах. Своего супруга Фернандес де Киршнер вспомнила и во время клятвы президента.\r\n\r\n"Я, Кристина Фернандес де Киршнер, клянусь Богом, Родиной и Священными Писаниями исполнять обязанности президента с верностью и патриотизмом, соблюдать Конституцию, а если я этого не сделаю, пусть Бог, Родина и он (Нестор Киршнер) спросят с меня за это", - сказала она.\r\n\r\nВо время присяги было видно, как сильно переживает президент, едва сдержая слезы при 

процедуры обработки взять из вебинарного ноутбука

In [25]:
# !pip install razdel

In [26]:
from razdel import tokenize

In [27]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [32]:
words[15:17]

[['Марину', 'PER'], ['Ковтун', 'PER']]

In [37]:
words_docs[29:30]

[['Андрей', 'PER']]

In [45]:
import nltk

nltk.download('words')
nltk.download('maxent_ne_chunker')
nltk.download('names')

[nltk_data] Downloading package words to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package names to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


True

In [49]:
{
    (' '.join(c[0] for c in chunk), chunk.label() ) 
    for chunk in nltk.ne_chunk(words_docs) 
    if hasattr(chunk, 'label') 
}

{('ABC News', 'ORGANIZATION'),
 ('AP', 'ORGANIZATION'),
 ('America', 'GPE'),
 ('American', 'GPE'),
 ('Bernard', 'ORGANIZATION'),
 ('British', 'GPE'),
 ('CSSD', 'ORGANIZATION'),
 ('Carry', 'PERSON'),
 ('Clinton', 'PERSON'),
 ('FBI Agents Association', 'ORGANIZATION'),
 ('HSBC', 'ORGANIZATION'),
 ('Huffington Post', 'ORGANIZATION'),
 ('IKEA', 'ORGANIZATION'),
 ('Israel', 'GPE'),
 ('John', 'PERSON'),
 ('John Huntsman Jr', 'PERSON'),
 ('John Murphy', 'PERSON'),
 ('John Roos', 'PERSON'),
 ('KSCM', 'ORGANIZATION'),
 ('Kalina', 'GPE'),
 ('MBA', 'ORGANIZATION'),
 ('MasterCard', 'ORGANIZATION'),
 ('McFarlane', 'ORGANIZATION'),
 ('Moscow', 'GPE'),
 ('NetJets', 'ORGANIZATION'),
 ('New York', 'GPE'),
 ('PetroVietnam', 'ORGANIZATION'),
 ('Richard Olson', 'PERSON'),
 ('Richard Williamson', 'PERSON'),
 ('Royal Bank', 'ORGANIZATION'),
 ('Russia', 'GPE'),
 ('SUN Group', 'ORGANIZATION'),
 ('TSB', 'ORGANIZATION'),
 ('Washington', 'GPE'),
 ('Washington Post', 'ORGANIZATION'),
 ('АСВ', 'ORGANIZATION'),
 ('

In [ ]:
# установка deeppavlov

!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

Found existing installation: tensorflow 2.5.0
Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
     |████████████████████████████████| 203 kB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 235 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 1.7 MB/s  eta 0:00:01
     |████████████████████████████████| 323 kB 11.7 MB/s eta 0:00:01
     |████████████████████████████████| 377 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 436 kB/s  eta 0:00:01
     |████████████████████████████████| 895 kB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 1.8 MB/s  eta 0:00:01
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23140 sha256=d28474f8be8b7c3d67623b85d57a5496b089e822521d6341f2444fd6411bd8af
  Stored in directory: /home/jovyan/.

In [ ]:
!python -m deeppavlov install squad_bert
!python -m deeppavlov install ner_ontonotes

In [52]:
import deeppavlov
from deeppavlov import configs, build_model

ModuleNotFoundError: No module named 'deeppavlov'

In [ ]:
deeppavlov_ner = build_model(configs.ner.ner_bert_ent_and_type_rus, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])

2021-04-07 09:56:01.170 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/kbqa/datasets/entity_and_type_detection_rus.pickle download because of matching hashes
2021-04-07 09:56:20.740 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/kbqa/models/ner_cq_rus.tar.gz download because of matching hashes
2021-04-07 09:56:27.228 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2021-04-07 09:56:27.635 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ent_and_type_rus/tag.dict]
2021-04-07 09:56:58.169 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ent_and_type_rus/model]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ent_and_type_rus/model


[[['Нью',
   '-',
   'Йорк',
   ',',
   'США',
   ',',
   '30',
   'апреля',
   '2020',
   ',',
   '01',
   ':',
   '01',
   '—',
   'REGNUM',
   'В',
   'администрации',
   'президента',
   'США',
   'Дональда',
   'Трампа',
   'планируют',
   'пройти',
   'все',
   'этапы',
   'создания',
   'вакцины',
   'от',
   'коронавируса',
   'в',
   'ускоренном',
   'темпе',
   'и',
   'выпустить',
   '100',
   'млн',
   'доз',
   'до',
   'конца',
   '2020',
   'года',
   ',',
   'передаёт',
   'агентство',
   'Bloomberg',
   'со',
   'ссылкой',
   'на',
   'осведомлённые',
   'источники']],
 array([[[9.71761167e-01, 2.64181718e-02, 1.82059826e-03],
         [9.60046053e-01, 3.84359621e-02, 1.51804090e-03],
         [9.21502233e-01, 7.51483515e-02, 3.34933866e-03],
         [9.92245018e-01, 6.89126039e-03, 8.63717811e-04],
         [9.87261832e-01, 1.17831351e-02, 9.55077179e-04],
         [9.96919036e-01, 2.74484046e-03, 3.36058962e-04],
         [9.97517467e-01, 2.30196840e-03, 1.80645322e